In [ ]:
import cv2
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


In [ ]:

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
ROOT_DIR


In [ ]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
#sys.path.append(os.path.join(ROOT_DIR, "/samples/coco/"))  # To find local version
#import coco


In [ ]:
sys.path.append(os.path.join(ROOT_DIR, ""))  # To find local version
from samples.coco import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
# single sign detect
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_road_sign.h5")

# detect signs by type
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_road_signs.h5")

# detect signs by type
# COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0024.h5")
# COCO_MODEL_PATH_ALL= os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0030.h5")
COCO_MODEL_PATH_ALL= os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0030_all_data_all_layers.h5")
# COCO_MODEL_PATH_ALL= os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0030_SN_data_head_layers.h5")
# COCO_MODEL_PATH_ALL= os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0020_SN_data_4_layers_0_001lr.h5")
# COCO_MODEL_PATH_ALL= os.path.join(ROOT_DIR, "mask_rcnn_road_signs_0027_SN_data_all_layers.h5")

# Download COCO trained weights from Releases if needed
# if not os.path.exists

(COCO_MODEL_PATH):
#     utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "Mask_RCNN/samples/roadsigns/signs_dataset/")

IMAGE_DIR = os.path.join(ROOT_DIR, "Mask_RCNN/samples/roadsigns/signs_dataset_01/")


In [ ]:
class InferenceConfigAll(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # single sign detect
    NUM_CLASSES = 1 + 1  # 1 Background + 1 Building
    
    # detect signs by type
    #NUM_CLASSES = 1 + 35  # 1 Background + 1 Building
    #IMAGE_MAX_DIM=320
    #IMAGE_MIN_DIM=320
configAll = InferenceConfigAll()
#config.display()


In [ ]:

# Create model object in inference mode.
model_all = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=configAll)

# Load weights trained on MS-COCO
model_all.load_weights(COCO_MODEL_PATH_ALL, by_name=True)
#model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[ "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
#model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[ "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])


In [ ]:

class_names_all = ['BG', 'road_sign']


In [ ]:

your_path = "D:/TEMP/_deeplearning/road_signs/_video_25_01_2020/out_image/VID_20200125_160419_01752.jpg"

# your_path = "D:/TEMP/_deeplearning/__from_kiev/1.jpg"


In [ ]:

size = os.path.getsize(your_path)
filename = os.path.basename(your_path)


In [ ]:
image_all = skimage.io.imread(your_path)
# Run detection
results_all = model_all.detect([image_all], verbose=1)

# Visualize results
r_all = results_all[0]
visualize.display_instances(image_all, r_all['rois'], r_all['masks'], r_all['class_ids'], class_names_all, r_all['scores'])
                   

In [ ]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image


In [ ]:
image_all = skimage.io.imread(your_path)
# Run detection
results_all = model_all.detect([image_all], verbose=1)

# Visualize results
r_all = results_all[0]
color_img = display_instances(image_all, r_all['rois'], r_all['masks'], r_all['class_ids'], class_names_all, r_all['scores'])


In [ ]:
from skimage import measure
from pycocotools import mask
from skimage.filters import threshold_otsu
from scipy.spatial import ConvexHull
import json


In [ ]:
import scipy.misc
import tensorflow as tf
import time
save_path = r'C:\Users\Administrator\Image-Classification-by-Keras-and-Tensorflow-master\Using Tensorflow\data\save'


In [ ]:
image_all = skimage.io.imread(your_path)


In [ ]:

i = 0
dim = (128, 128)
for val in r_all['rois']:
    im_croped = []
    # print(val[0], val[2], val[1], val[3])

    # ---------------------------------------------
    # Идея вырезать картинки не по ббокс а по контуру маски. 
    # Будет ли так работать предсказание точнее?
    
    masks = r_all['masks'][:, :, i]
    ground_truth_binary_mask = masks
    contours = measure.find_contours(ground_truth_binary_mask, 0.5)
    hull = ConvexHull(contours[0])
    polygon =[]
    for v in hull.vertices:
        pt = []
        # print(contours[0][v][0], contours[0][v][1])
        pt.append(int(contours[0][v][0]))
        pt.append(int(contours[0][v][1]))
        polygon.append(pt)
    pts = np.array(polygon)
    # print (pts)
    rect = cv2.boundingRect(pts)
    y, x, w, h = rect
    croped = image_all[y:y+w, x:x+h].copy()
    pts = pts - pts.min(axis=0)
    pts = np.flip(pts,1)
    # print (pts)

    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

    ## (3) do bit-op
    dst = cv2.bitwise_and(croped, croped, mask=mask)

    ## (4) add the white background
    bg = np.ones_like(croped, np.uint8) * 255
    cv2.bitwise_not(bg, bg, mask=mask)
    dst2 = bg + dst
    dim = (128, 128)
    dst2 = cv2.resize(dst2, dim, interpolation=cv2.INTER_AREA)
    fig, ax = plt.subplots()
    ax.imshow(dst2)
    im_path = save_path + '\image_' + str(i) + '.png'
    scipy.misc.imsave(im_path, dst2)
    
    # ---------------------------------------------
    # Старый код
    
    # cropped = image_all[val[0]:val[2],val[1]:val[3]]
    # fig, ax = plt.subplots()
    # ax.imshow(cropped)
    # im_path = save_path + '\image_' + str(i) + '.png'
    # cropped = cv2.resize(cropped, dim, interpolation=cv2.INTER_AREA)
    # scipy.misc.imsave(im_path, cropped)
    
    i = i + 1
    


In [ ]:

    start = time.time()

# try:
    
# Path of  training images
train_path = r'C:\Users\Administrator\Image-Classification-by-Keras-and-Tensorflow-master\Using Tensorflow\data\train'
if not os.path.exists(train_path):
    print("No such directory")
    raise Exception
# Path of testing images
dir_path = r'C:\Users\Administrator\Image-Classification-by-Keras-and-Tensorflow-master\Using Tensorflow\data\save'
if not os.path.exists(dir_path):
    print("No such directory")
    raise Exception

# Walk though all testing images one by one
for root, dirs, files in os.walk(dir_path):
    for name in files:

        print("")
        image_path = name
        filename = dir_path +'\\' +image_path
        print(filename)
        image_size=128
        num_channels=3
        images = []

        if os.path.exists(filename):

            # Reading the image using OpenCV
            image = cv2.imread(filename)
            # Resizing the image to our desired size and preprocessing will be done exactly as done during training
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            images.append(image)
            images = np.array(images, dtype=np.uint8)
            images = images.astype('float32')
            images = np.multiply(images, 1.0/255.0)

            # The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
            x_batch = images.reshape(1, image_size,image_size,num_channels)

            # Let us restore the saved model
            sess = tf.Session()
            # Step-1: Recreate the network graph. At this step only graph is created.
            saver = tf.train.import_meta_graph('C:/Users/Administrator/Image-Classification-by-Keras-and-Tensorflow-master/Using Tensorflow/models/model.meta')
            # Step-2: Now let's load the weights saved using the restore method.
            saver.restore(sess, tf.train.latest_checkpoint('C:/Users/Administrator/Image-Classification-by-Keras-and-Tensorflow-master/Using Tensorflow/models/'))

            # Accessing the default graph which we have restored
            graph = tf.get_default_graph()

            # Now, let's get hold of the op that we can be processed to get the output.
            # In the original network y_pred is the tensor that is the prediction of the network
            y_pred = graph.get_tensor_by_name("y_pred:0")

            ## Let's feed the images to the input placeholders
            x= graph.get_tensor_by_name("x:0")
            y_true = graph.get_tensor_by_name("y_true:0")
            y_test_images = np.zeros((1, len(os.listdir(train_path))))


            # Creating the feed_dict that is required to be fed to calculate y_pred
            feed_dict_testing = {x: x_batch, y_true: y_test_images}
            result=sess.run(y_pred, feed_dict=feed_dict_testing)
            # Result is of this format [[probabiliy_of_classA probability_of_classB ....]]
            print(result)

            # Convert np.array to list
            a = result[0].tolist()
            r=0

            # Finding the maximum of all outputs
            max1 = max(a)
            index1 = a.index(max1)
            predicted_class = None

            # Walk through directory to find the label of the predicted output
            count = 0
            for root, dirs, files in os.walk(train_path):
                for name in dirs:
                    if count==index1:
                        predicted_class = name
                    count+=1

            # If the maximum confidence output is largest of all by a big margin then
            # print the class or else print a warning
            for i in a:
                if i!=max1:
                    if max1-i<i:
                        r=1
            if r ==0:
                print(predicted_class)
            else:
                print("Could not classify with definite confidence")
                # print("Maybe:",predicted_class)

        # If file does not exist
        else:
            print("File does not exist")
                
# except Exception as e:
#     print("Exception:",e)

# Calculate execution time
end = time.time()
dur = end-start
print("")
if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")
    
    
